In [5]:
import pandas as pd
import numpy as np

# combined data - 2 tables

In [71]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [72]:
filename_leadercareerlink = "leadercareerlink.xlsx"

In [73]:
filename_leaderjoblink = "leaderjoblink.xlsx"

In [74]:
filename_joborglink = "joborglink.xlsx"

# combined data - 1 cleaning/cleaning step 2 - career/

In [75]:
path_cleaning = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/"

In [76]:
subpath_2_1 = "2.1 career_undivided_unparsed_uncoded/"
subpath_2_2 = "2.2 career_divided_unparsed_uncoded/"
subpath_2_3 = "2.3 joborglink/"

In [77]:
filein_careerlink = "career_link_undivided_unparsed_uncoded.xlsx"
careerlink = pd.read_excel(path_cleaning + subpath_2_1 + filein_careerlink)
careerlink.shape

(12985, 4)

In [78]:
filein_careerjobs = "career_jobs_unparsed_uncoded.xlsx"
careerjobs = pd.read_excel(path_cleaning + subpath_2_2 + filein_careerjobs)
careerjobs.shape

(7590, 3)

In [113]:
filein_multiple = "joborglink_1_mergeold_mail.xlsx"
jol1 = pd.read_excel(path_cleaning + subpath_2_3 + filein_multiple)
jol1.shape

(7592, 14)

In [79]:
filein_jol_7 = "joborglink_7_2.xlsx"
jol7 = pd.read_excel(path_cleaning + subpath_2_3 + filein_jol_7)
jol7.shape

(1425, 4)

# 1. leadercareerlink 

In [80]:
# from career_link_undivided_unparsed_uncoded
careerlink.head(3)

,LeaderID,CareerStringID_2015,CareerString,CareerDateString_2022
0,리선권,NaN,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,NaN,故 김정일 국가장의위원회 위원,2011.12
2,리영수,NaN,근로단체부장 해임 * 후임: 리일환,2014


In [81]:
select_columns = ["LeaderID","CareerString","CareerDateString_2022"]
careerlink = careerlink[select_columns]

In [82]:
careerlink[careerlink.duplicated(keep=False)]

,LeaderID,CareerString,CareerDateString_2022
241,김일본,1950.12 민족보위성 부상 겸 문화부 부장,NaN
242,김일본,1950.12 민족보위성 부상 겸 문화부 부장,NaN
247,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
248,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
249,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
...,...,...,...
5468,현영철,"2012. 7 인민군 총참모장, 인민군 차수, 당 중앙군사위원회 부위원장",NaN
5488,리철만,2013. 4 내각 부총리 겸 농업상,NaN
5489,리철만,2013. 4 내각 부총리 겸 농업상,NaN
5490,리무영,2013. 4 내각 부총리 겸 화학공업상,NaN


In [83]:
leadercareerlink = careerlink.drop_duplicates(keep="first")
leadercareerlink.shape

(12617, 3)

In [84]:
leadercareerlink.head(5)

,LeaderID,CareerString,CareerDateString_2022
0,리선권,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,故 김정일 국가장의위원회 위원,2011.12
2,리영수,근로단체부장 해임 * 후임: 리일환,2014
3,리설주,금수산궁전 개관식 참석시 동행,2013.08
4,리원일,노동성 상(유임),1999.02


In [85]:
# export leadercareerlink to tables
# leadercareerlink.to_excel(path_tables+filename_leadercareerlink,index=False)

# 2. joborglink

### 2.1 - careers - isolate careers from leadercareerlink

In [86]:
select_columns = ["CareerString","CareerDateString_2022"]
careers = leadercareerlink[select_columns]
careers.shape

(12617, 2)

In [87]:
careers[careers.duplicated(keep=False)].shape

(5125, 2)

In [88]:
careers = careers.drop_duplicates(keep="first")
careers.shape

(8045, 2)

In [89]:
careers[careers.duplicated(["CareerString","CareerDateString_2022"],keep=False)]

,CareerString,CareerDateString_2022


### 2.2 - careers0 - add IsJob
- compare careers with career_jobs_unparsed_uncoded
- add IsJob=False for all rows
- recode IsJob=True for all CareerString contained in career_jobs_unparsed_uncoded
- don't code CareerStringType - because we still have MultipleSubstrings
- just note that all career_jobs_unparsed_uncoded are contained in careers
- later, use the files in subpath_2_2 = "2.2 career_divided_unparsed_uncoded/" to help code CareerSubstringType

In [90]:
# compare careers with careerjobs
careerjobs.head(3)

,CareerString,CareerDateString_2022,CareerStringID
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,1
1,근로단체부장 해임 * 후임: 리일환,2014,2
2,노동성 상(유임),1999.02,4


In [91]:
select_columns = ["CareerString","CareerDateString_2022"]
careerjobs = careerjobs[select_columns]
careerjobs.shape

(7590, 2)

In [92]:
careerjobs[careerjobs.duplicated(keep=False)]

,CareerString,CareerDateString_2022


In [93]:
careers.head(3)

,CareerString,CareerDateString_2022
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,故 김정일 국가장의위원회 위원,2011.12
2,근로단체부장 해임 * 후임: 리일환,2014


In [94]:
# merge and compare careers & careerjobs
select_columns = ["CareerString","CareerDateString_2022"]
careers0 = careers.merge(careerjobs,on=select_columns,how="outer",indicator=True)

In [95]:
careers0.head(3)

,CareerString,CareerDateString_2022,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,both
1,故 김정일 국가장의위원회 위원,2011.12,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,both


In [96]:
careers0[careers0["_merge"]=="left_only"].shape

(455, 3)

In [97]:
careers0[careers0["_merge"]=="both"].shape

(7590, 3)

In [98]:
careers0[careers0["_merge"]=="right_only"].shape

(0, 3)

In [99]:
careers0[careers0["_merge"]=="right_only"]

,CareerString,CareerDateString_2022,_merge


In [104]:
careers0["IsJob"] = False

In [106]:
careers0.loc[careers0["_merge"]=="both","IsJob"] = True

In [111]:
careers0.groupby("IsJob").count()

,CareerString,CareerDateString_2022,_merge
IsJob,,,
False,455,259,455
True,7590,4209,7590


In [119]:
select_columns = ["CareerString","CareerDateString_2022","IsJob"]
careers0 = careers0[select_columns]

In [121]:
careers0.head(3)

,CareerString,CareerDateString_2022,IsJob
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,True
1,故 김정일 국가장의위원회 위원,2011.12,False
2,근로단체부장 해임 * 후임: 리일환,2014,True


### 2.3 - careers1 - add MultipleString
- compare careers1 with jol1 (joborglink_1_mergeold_mail)
- update IsJob with jol1["NotJob"]
- add MultipleSubstrings
- code MultipleSubstrings

In [117]:
select_columns = ["CareerString","CareerDateString_2022","NotJob","MultipleSubstrings"]
jol1 = jol1[select_columns]
jol1.head(3)

,CareerString,CareerDateString_2022,NotJob,MultipleSubstrings
0,1981.10 당 정치국 후보위원으로 강등,NaN,0.0,NaN
1,1990. 5 당 정치국 후보위원으로 강등,NaN,0.0,NaN
2,1991.12 인민군 최고사령관직을 김정일에게 이양,NaN,0.0,NaN


In [150]:
jol1 = jol1.drop_duplicates(keep="first")
jol1.shape

(7521, 4)

In [153]:
# no duplicate keys in jol1
select_key = ["CareerString","CareerDateString_2022"]
jol1[jol1.duplicated(select_key,keep=False)]

,CareerString,CareerDateString_2022,NotJob,MultipleSubstrings


In [157]:
careers0.shape

(8045, 3)

In [154]:
select_key = ["CareerString","CareerDateString_2022"]
careers1 = careers0.merge(jol1,on=select_key,how="outer",indicator=True)
careers1.shape

(11748, 6)

In [155]:
careers1.head(3)

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,True,NaN,NaN,left_only
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN,NaN,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,True,NaN,NaN,left_only


In [156]:
# IsJob = 
careers1.groupby(["IsJob","NotJob"],dropna=False).count()

CareerString  CareerDateString_2022  MultipleSubstrings  _merge
IsJob NotJob                                                                 
False NaN              455                    259                   0     455
True  0.0              415                    232                   0     415
      1.0              300                     34                   0     300
      NaN             6875                   3943                 329    6875
NaN   0.0                2                      0                   0       2
      1.0              203                    171                   0     203
      NaN             3498                   3389                 408    3498

In [135]:
careers1[careers1["_merge"]=="left_only"].shape

(4227, 6)

In [136]:
careers1[careers1["_merge"]=="both"].shape

(3842, 6)

In [137]:
careers1[careers1["_merge"]=="right_only"].shape

(3750, 6)

In [139]:
careers1_left = careers1[careers1["_merge"]=="left_only"]

In [141]:
careers1_left.groupby(["IsJob","NotJob"],dropna=False).count()

,,CareerString,CareerDateString_2022,MultipleSubstrings,_merge
IsJob,NotJob,,,,
False,NaN,455,259,0,455
True,NaN,3772,3618,0,3772


In [145]:
careers1_left[careers1_left["IsJob"]==True]

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,True,NaN,NaN,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,True,NaN,NaN,left_only
4,노동성 상(유임),1999.02,True,NaN,NaN,left_only
6,당 정치국 후보위원,2020.01,True,NaN,NaN,left_only
7,당 중앙위원회 정치국 위원 (*당 중앙위원회 제7기 제4차 전원회의에서 보선),2019.04,True,NaN,NaN,left_only
...,...,...,...,...,...,...
8002,호위사령관 해임,2004,True,NaN,NaN,left_only
8012,화성의숙(길림성 화전현) 전학,1926,True,NaN,NaN,left_only
8039,"황해북도 당 위원회 위원장, 당 중앙위원회 위원",2017.10,True,NaN,NaN,left_only
8052,황해북도 행정경제위원회 위원장(해임),1994.05,True,NaN,NaN,left_only


In [142]:
careers1_both = careers1[careers1["_merge"]=="both"]

In [143]:
careers1_both.groupby(["IsJob","NotJob"],dropna=False).count()

CareerString  CareerDateString_2022  MultipleSubstrings  _merge
IsJob NotJob                                                                 
True  0.0              415                    232                   0     415
      1.0              300                     34                   0     300
      NaN             3127                    349                 329    3127